In [1]:
!pip install weaviate-client
!pip install langchain_groq
!pip -q install langchain langchain_community

In [2]:
from langchain_groq import ChatGroq

In [3]:
from google.colab import userdata

In [4]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = userdata.get('HF_TOKEN')
groq_api_key=userdata.get("groq_api")

In [5]:
WEAVIATE_API_KEY="W7VkHSTI45CG0UHovKo4YLaQRYRn1H2QgDWh"
WEAVIATE_CLUSTER="https://pj9ida1ty6v6nxpqvyxlq.c0.us-east1.gcp.weaviate.cloud"

# Data Reading

In [6]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [7]:
!pip install "unstructured[pdf]"

In [8]:
from langchain.document_loaders import DirectoryLoader

loader=DirectoryLoader("./data",glob="**/*.pdf")
data=loader.load()

In [9]:
data

[Document(metadata={'source': 'data/yolov7paper.pdf'}, page_content='2 2 0 2\n\nl u J\n\n6\n\n]\n\nV C . s c [\n\n1 v 6 9 6 2 0 . 7 0 2 2 : v i X r a\n\nYOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object detectors\n\nChien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1 1Institute of Information Science, Academia Sinica, Taiwan kinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw\n\nAbstract\n\nYOLOv7 surpasses all known object detectors in both speed and accuracy in the range from 5 FPS to 160 FPS and has the highest accuracy 56.8% AP among all known real-time object detectors with 30 FPS or higher on GPU V100. YOLOv7-E6 object detector (56 FPS V100, 55.9% AP) outperforms both transformer-based detector SWIN- L Cascade-Mask R-CNN (9.2 FPS A100, 53.9% AP) by 509% in speed and 2% in accuracy, and convolutional- based detector ConvNeXt-XL Cascade-Mask R-CNN (8.6 FPS A100, 55.2% AP) by 551% in speed and 0.7% AP in accuracy, as we

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
docs=text_splitter.split_documents(data)

In [13]:
len(docs)

90

# Embedding Convertion

In [15]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [17]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Vector Database Initialization

In [18]:
import weaviate
from langchain.vectorstores import Weaviate

# Connect to weaviate Cluster
auth_config=weaviate.auth.AuthApiKey(api_key=WEAVIATE_API_KEY)
WEAVIATE_URL=WEAVIATE_CLUSTER

In [23]:
client=weaviate.Client(
    url=WEAVIATE_URL,
    additional_headers={'GROQ_API_KEY':groq_api_key},
      auth_client_secret=auth_config,
      startup_period=10
)

<ipython-input-23-f6c81114e6dd>:1: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client=weaviate.Client(


In [ ]:
client.is_ready()

In [ ]:
# define input structure
client.schema.delete_all()
client.schema.get()
schema = {
    "classes": [
        {
            "class": "Chatbot",
            "description": "Documents for chatbot",
            "vectorizer": "text2vec-openai",
            "moduleConfig": {"text2vec-openai": {"model": "ada", "type": "text"}},
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "moduleConfig": {
                        "text2vec-openai": {
                            "skip": False,
                            "vectorizePropertyName": False,
                        }
                    },
                    "name": "content",
                },
            ],
        },
    ]
}

client.schema.create(schema)
vectorstore = Weaviate(client, "Chatbot", "content", attributes=["source"])


In [ ]:
text_meta_pair = [(doc.page_content, doc.metadata) for doc in docs]
texts, meta = list(zip(*text_meta_pair))
vectorstore.add_texts(texts, meta)

In [ ]:
query = "what is a yolo?"

# retrieve text related to the query
docs = vectorstore.similarity_search(query, top_k=20)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
# define chain
chain = load_qa_chain(
    OpenAI(openai_api_key = OPENAI_API_KEY,temperature=0),
    chain_type="stuff")

In [ ]:
# create answer
chain.run(input_documents=docs, question=query)